# Performance considerations with python

Python is a relatively easy language to learn (if you don't go deep into the weeds), and it is by far the dominant 
language in any kind of scientific realm, including data science.  Unfortunately, python's performance leaves a lot to
be desired.  Alternative interpreters like pypy or graalpy, while faster, are also quite a bit behind (3.10 for pypy
and 3.11 for graalpy).

Python 3.11 came out with up to 60% performance improvements over 3.10, and on average is about 25% faster.  Initial
benching of 3.12 are giving about a 10% performance boost compared to 3.11.  While that's admirable, it still leaves
python way behind even javascript's performance.

So is python doomed to poor performance?  This document will cover ways to boost python performance in common tasks,
and to separate IO vs CPU bound tasks.  We'll also spend some time with accelerating data engineering tasks and looking
at how GPU compute works.

- Cncurrency 
    - asyncio: IO bound performance
    - multiprocessing: CPU bound per process
    - multithreading: GIL (and no GIL)
    - subinterpreters
- Profiler
- Data structures
- Alternative compilers or JITs
    - mypyc
    - cython
    - numba
    - triton
    - mojo (BSL)
    - codon (BSL)
- Rust Extensions (pyo3)

## Using async to boost IO performance

An extremely common task is downloading something, typically from a web service or S3.  This is a great example of an
IO bound task.  Generally speaking, a task (a function call or group of related function calls) can be thought of as
either IO or CPU bound.  An IO bound task spends the majority of it's time doing some kind of IO work; downloading a 
file, waiting for an HTTP response, reading or writing a file on disk, etc etc.  A CPU bound spends the majority of its
time in a CPU intensive task; calculating a matrix multiplication, sorting a large data structure based on some
predicate, or some other calculation.

But why is an IO bound task _bound_?  What does it mean to be _bound_?  In most libraries and frameworks (including the
standard library of the programming language) they use _synchronous_ IO methods.  What that means is that when you make
a call for some IO related function (eg, reading from disk, making a network send call, etc) your code _blocks_ until
there is a response back.  This response may come from your own system, such as when you read from disk, or from a 
remote computer, such as when you make an HTTP GET request.  Basically your code (technically, the CPU) has to wait
until it receives a response.

In contrast to this are _async_ IO methods.  Under the hood, they use a different set of OS system calls that are
**not** blocking.  Instead, these IO calls return immediately.  From this point, how the high level async library, the
programming language it is written in and the low-level OS calls interact is highly variable.

For example, javascript uses the notion of a `Promise` to indicate that it promises that an answer will _eventually_
be available (even if the answer is `null` or `undefined`).  Java uses a `Future` and rust also uses a `Future` except
that a rust `Future` is implemented very differently from a Java one.  In python, it too has a `Future` and a `Task`
and these represent the concept that eventually, you will get an answer from the async method.

How can this be faster than multithreading at times?  Even if python did not have a Global Interpreter Lock (GIL), using
multiple threads is not a good solution for many IO bound tasks.  The classic example of this (and what led to async
becoming a huge requirement...even to the creation of the node language) is the 10k problem.  Basically, this was the
problem of a web service trying to handle 10k+ simultaneous connections.

When your service becomes very popular, it may take more than a single computer to handle all the requests.  There's a
bit of hidden complexity in the term _handle_.  There's actually two parts to this: 

- accepting the request, including any parameters, and sending some kind of request back to the caller
- doing the work that was specified in the request (eg, performing a query, downloading a resource, etc)

As seen from _handling_ a request, if part 1 is acknowledging the request, extracting the parameters, and returning
something back to the caller, we have 2 choices.  The service 
